# Primeiro dataset

Dataset escolhido: [https://archive.ics.uci.edu/dataset/602/dry+bean+dataset](https://archive.ics.uci.edu/dataset/602/dry+bean+dataset)

## Extraindo os dados

In [28]:
import pandas as pd

path_to_data = 'data/Dry_Bean_Dataset.xlsx'

# Carregar base de dados
# DataFrame
HEADER_ROW = 0
dataset = pd.read_excel(path_to_data, sheet_name="Dry_Beans_Dataset",header=HEADER_ROW)

# DEBUG - Verificar se a base de dados foi carregada corretamente
# dataset.head(20)
# dataset.shape

## Transformando os dados

In [29]:
# Transformando as labels em valores numéricos
dataset['Class'] = dataset['Class'].astype('category').cat.codes

## Separando os dados
Seperação dos dados em features (X) e target (y)

In [30]:
NUM_OF_COLUMNS = len(dataset.columns)

y = dataset.iloc[:,-1] # extrai o target (última coluna)
X = dataset.iloc[:,0:NUM_OF_COLUMNS-1] # extrai as features (todas as colunas menos a última)

## Normalizando os dados
Deixando os valores de todas as colunas entre 0 e 1

In [31]:
from sklearn.preprocessing import MinMaxScaler

x_values = X.values
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x_values)
X = pd.DataFrame(x_scaled)

## Criando os folds
Serão 10 folds para o k-fold cross validation

In [32]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Transforma para array NumPy
X = np.array(X)
y = np.array(y)

# Define o número de folds
N_FOLDS = 10
kf = StratifiedKFold(n_splits=N_FOLDS)

# features e target de treino
X_train = []
y_train = []

# features e target de teste
X_test = []
y_test = []

for train_index, test_index in kf.split(X, y):
  X_train.append(X[train_index])
  X_test.append(X[test_index])

  y_train.append(y[train_index])
  y_test.append(y[test_index])

## Obtendo os scores dos modelos

In [33]:
from typing import List, Any
from sklearn import metrics

def predict_and_get_accuracy(model, X_test: List[Any], y_test: List[Any]) -> float:
  y_pred = model.predict(X_test)
  accuracy = metrics.accuracy_score(y_test, y_pred)

  return accuracy

### Árvore de decisão

In [34]:
from sklearn.tree import DecisionTreeClassifier

gini_tree_results = []
entropy_tree_results = []

TREE_RANDOM_STATE = 42
TREE_MAX_DEPTH = 5

# Treinamento e avaliação do modelo com critério entropy
for i in range(N_FOLDS):
  entropy_tree_model = DecisionTreeClassifier(criterion="entropy", random_state=TREE_RANDOM_STATE,
    max_depth=TREE_MAX_DEPTH)
  entropy_tree_model = entropy_tree_model.fit(X_train[i], y_train[i])

  entropy_tree_results.append(predict_and_get_accuracy(entropy_tree_model, X_test[i], y_test[i]))

print(f"Acurácia média da árvore de decisão com ENTROPY: { \
  round(np.mean(entropy_tree_results)*100)}%")

# Treinamento e avaliação do modelo com critério gini
for i in range(N_FOLDS):
  gini_tree_model = DecisionTreeClassifier(criterion="gini", random_state=TREE_RANDOM_STATE,
    max_depth=TREE_MAX_DEPTH)
  gini_tree_model = gini_tree_model.fit(X_train[i], y_train[i])

  gini_tree_results.append(predict_and_get_accuracy(gini_tree_model, X_test[i], y_test[i]))

print(f"Acurácia média da árvore de decisão com GINI: { \
  round(np.mean(gini_tree_results)*100)}%")

Acurácia média da árvore de decisão com ENTROPY: 80%
Acurácia média da árvore de decisão com GINI: 74%


### K-Nearest Neighbors (KNN)

In [35]:
from sklearn.neighbors import KNeighborsClassifier

five_neighbors_results = []
ten_neighbors_results = []

# Treinamento e avaliação do modelo com 5 vizinhos
for i in range(N_FOLDS):
  five_neighbors_model = KNeighborsClassifier(n_neighbors=5, metric="euclidean")
  five_neighbors_model = five_neighbors_model.fit(X_train[i], y_train[i])

  five_neighbors_results.append(predict_and_get_accuracy(five_neighbors_model, X_test[i], y_test[i]))

print(f"Acurácia média do KNN com 5 vizinhos: { \
  round(np.mean(five_neighbors_results)*100)}%")

# Treinamento e avaliação do modelo com 10 vizinhos
for i in range(N_FOLDS):
  ten_neighbors_model = KNeighborsClassifier(n_neighbors=10, metric="euclidean")
  ten_neighbors_model = ten_neighbors_model.fit(X_train[i], y_train[i])

  ten_neighbors_results.append(predict_and_get_accuracy(ten_neighbors_model, X_test[i], y_test[i]))

print(f"Acurácia média do KNN com 10 vizinhos: { \
  round(np.mean(ten_neighbors_results)*100)}%")

Acurácia média do KNN com 5 vizinhos: 88%
Acurácia média do KNN com 10 vizinhos: 88%


### Multilayer Perceptron (MLP)

In [36]:
from sklearn.neural_network import MLPClassifier

mlp_first_architecture_results = {
    "tanh": [],
    "relu": [],
}

mlp_second_architecture_results = {
    "tanh": [],
    "relu": [],
}

# Treinamento e avaliação do modelo com função de ativação RELU (arquitetura 15, 2)
for i in range(N_FOLDS):
  mlp_first_architecture_model = MLPClassifier(hidden_layer_sizes=(10, 2,), random_state=42,
    max_iter=1000)
  mlp_first_architecture_model = mlp_first_architecture_model.fit(X_train[i], y_train[i])

  mlp_first_architecture_results["relu"].append(
    predict_and_get_accuracy(mlp_first_architecture_model, X_test[i], y_test[i]))
  
print(f"Acurácia média da MLP com arquitetura (10, 2) e função de ativação RELU: { \
  round(np.mean(mlp_first_architecture_results['relu'])*100)}%")

# Treinamento e avaliação do modelo com função de ativação RELU (arquitetura 3,)
for i in range(N_FOLDS):
  mlp_second_architecture_model = MLPClassifier(hidden_layer_sizes=(50,), random_state=42,
    max_iter=1000)
  mlp_second_architecture_model = mlp_second_architecture_model.fit(X_train[i], y_train[i])

  mlp_second_architecture_results["relu"].append(
    predict_and_get_accuracy(mlp_second_architecture_model, X_test[i], y_test[i]))

print(f"Acurácia média da MLP com arquitetura (3, 2) e função de ativação RELU: { \
  round(np.mean(mlp_second_architecture_results["relu"])*100)}%")

Acurácia média da MLP com arquitetura (10, 2) e função de ativação RELU: 88%
Acurácia média da MLP com arquitetura (3, 2) e função de ativação RELU: 89%
